In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("merged.csv")

C:\Users\kubic\AppData\Local\Temp\ipykernel_3512\1330870347.py:1: DtypeWarning: Columns (2,22,41,49,59,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("merged.csv")


In [3]:
drop_cols = [
    # IDs / URLs
    'id', 'listing_url', 'scrape_id', 'picture_url',
    'host_id', 'host_url', 'host_thumbnail_url', 'host_picture_url',

    # Free text (NLP not in course)
    'name', 'description', 'neighborhood_overview', 'host_about', 'amenities',

    # Dates (unless feature-engineered, which you didn't cover)
    'last_scraped', 'host_since', 'calendar_last_scraped',
    'first_review', 'last_review',

    # Administrative / rarely informative
    'calendar_updated', 'license',

    # Unimportant info
    'host_location','source','host_name','host_neighbourhood','host_verifications'
]

df = df.drop(columns=drop_cols)


In [4]:
# Keep only numeric columns + city + target
cols_to_keep = (
    df.select_dtypes(include=['int64', 'float64']).columns
    .tolist()
)

# Ensure target and city are kept
if 'price_local' not in cols_to_keep:
    cols_to_keep.append('price_local')

cols_to_keep.append('city')

# Drop everything else
df_test = df[cols_to_keep]

print("Remaining columns:", df_test.shape[1])
print(df_test.columns)


Remaining columns: 39
Index(['host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'price_local', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'review_scores_rating',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_

In [6]:
df_oslo = df_test[df_test['city'] == 'Oslo']
df_cph  = df_test[df_test['city'] == 'Copenhagen']

In [11]:
def run_models(df_city):
    X = df_city.drop(columns=['price_local', 'city'])
    y = df_city['price_local']

    # Force numeric (non-numeric -> NaN)
    X = X.apply(pd.to_numeric, errors='coerce')

    # 1) Drop columns that are entirely NaN
    X = X.dropna(axis=1, how='all')

    # 2) Median-impute remaining NaNs
    X = X.fillna(X.median())

    # 3) Safety: if anything is STILL NaN, drop those columns too
    X = X.loc[:, ~X.isna().any(axis=0)]

    # Drop rows with missing y (just in case)
    mask_y = y.notna()
    X = X[mask_y]
    y = y[mask_y]

    # Final sanity check
    assert X.shape[0] > 0, "No rows left after cleaning"
    assert X.shape[1] > 0, "No features left after cleaning"
    assert not np.isnan(X.to_numpy()).any(), "Still NaNs in X"

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    ridge = RidgeCV(alphas=np.logspace(-3, 3, 50), cv=5)
    lasso = LassoCV(alphas=np.logspace(-3, 1, 50), cv=5, max_iter=20000)

    ridge.fit(X_scaled, y)
    lasso.fit(X_scaled, y)

    return ridge, lasso, X.columns
ridge_oslo, lasso_oslo, feat_oslo = run_models(df_oslo)
ridge_cph,  lasso_cph,  feat_cph  = run_models(df_cph)

c:\Users\kubic\OneDrive\Dokumenty\envs\lsda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 183650684.03962708, tolerance: 3036287.5899855937
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\kubic\OneDrive\Dokumenty\envs\lsda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1390405198.0896835, tolerance: 3036287.5899855937
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\kubic\OneDrive\Dokumenty\envs\lsda\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2770330548.4622955, tolerance: 3036287.5899855937
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\kubic\OneDrive\Dokumen

In [12]:
print("Oslo:")
print("  Ridge non-zero:", np.sum(ridge_oslo.coef_ != 0))
print("  Lasso non-zero:", np.sum(lasso_oslo.coef_ != 0))

print("\nCopenhagen:")
print("  Ridge non-zero:", np.sum(ridge_cph.coef_ != 0))
print("  Lasso non-zero:", np.sum(lasso_cph.coef_ != 0))

Oslo:
  Ridge non-zero: 36
  Lasso non-zero: 30

Copenhagen:
  Ridge non-zero: 36
  Lasso non-zero: 20


In [15]:
lasso_oslo_coef = pd.Series(lasso_oslo.coef_, index=feat_oslo)
lasso_oslo_selected = lasso_oslo_coef[lasso_oslo_coef != 0] \
    .sort_values(key=np.abs, ascending=False)

lasso_oslo_selected.head(30)

maximum_maximum_nights                          7407.800193
maximum_nights_avg_ntm                         -3837.927201
minimum_maximum_nights                         -3526.274943
host_total_listings_count                       1161.874045
estimated_occupancy_l365d                       -751.439716
estimated_revenue_l365d                          730.446206
calculated_host_listings_count                  -334.664722
bathrooms                                        286.712378
bedrooms                                         221.979137
availability_30                                  170.625069
calculated_host_listings_count_private_rooms    -140.545704
accommodates                                      78.562744
maximum_nights                                    76.062318
minimum_minimum_nights                            73.172042
beds                                              72.497515
longitude                                        -67.519346
latitude                                

In [16]:
lasso_cph_coef = pd.Series(lasso_cph.coef_, index=feat_cph)
lasso_cph_selected = lasso_cph_coef[lasso_cph_coef != 0] \
    .sort_values(key=np.abs, ascending=False)

lasso_cph_selected.head(30)

estimated_revenue_l365d                         676.280499
estimated_occupancy_l365d                      -489.150739
minimum_minimum_nights                          201.572631
minimum_maximum_nights                          175.297525
bedrooms                                        139.241668
accommodates                                    137.269407
availability_60                                 120.366716
distance_to_center_km                          -119.233901
host_total_listings_count                       107.440777
maximum_maximum_nights                          -97.877327
bathrooms                                        87.132747
calculated_host_listings_count_entire_homes      81.878836
availability_90                                  79.809694
longitude                                        48.024514
maximum_nights                                  -44.793853
review_scores_cleanliness                        32.832235
calculated_host_listings_count_private_rooms    -29.2434

In [17]:
oslo_feats = set(lasso_oslo_selected.index)
cph_feats  = set(lasso_cph_selected.index)

common = oslo_feats & cph_feats
only_oslo = oslo_feats - cph_feats
only_cph  = cph_feats - oslo_feats

print("Common features:", common)
print("Only Oslo:", only_oslo)
print("Only Copenhagen:", only_cph)


Common features: {'calculated_host_listings_count_private_rooms', 'distance_to_center_km', 'minimum_minimum_nights', 'longitude', 'minimum_maximum_nights', 'accommodates', 'host_total_listings_count', 'review_scores_cleanliness', 'estimated_occupancy_l365d', 'maximum_maximum_nights', 'maximum_nights', 'number_of_reviews_ly', 'bedrooms', 'estimated_revenue_l365d', 'availability_90', 'latitude', 'bathrooms', 'review_scores_location'}
Only Oslo: {'maximum_nights_avg_ntm', 'availability_30', 'minimum_nights_avg_ntm', 'calculated_host_listings_count', 'beds', 'calculated_host_listings_count_shared_rooms', 'maximum_minimum_nights', 'review_scores_checkin', 'review_scores_communication', 'review_scores_rating', 'review_scores_value', 'availability_365'}
Only Copenhagen: {'availability_60', 'calculated_host_listings_count_entire_homes'}
